# Behavior Cloning

### Data Augmentation and Formatting
Flip images and measurements

In [1]:
import csv
import cv2
import numpy as np

In [2]:
lines = []
with open("./data/driving_log.csv") as f:
    reader = csv.reader(f)
    for line in reader:
        lines.append([line[0], *line[-4:]])

fw = open("./data/driving_log_updated.csv", "w")
for line in lines:
    img = cv2.imread(line[0], cv2.IMREAD_COLOR)
    img = np.fliplr(img)
    new_filename = line[0][:-4]+"_flipped.jpg"
    fw.write("{},{},{},{},{}\n".format(*line))
    fw.write("{},{},{},{},{}\n".format(new_filename, -1.0 * float(line[1]), -1.0 * float(line[2]), -1.0 * float(line[3]), -1.0 * float(line[4])))
    cv2.imwrite(new_filename, img)
    
fw.close()

KeyboardInterrupt: 

Crop Images

In [22]:
# 55 off top
# 30 off bottom
fw = open("./data/driving_log_updated_cropped.csv", "w")
with open("./data/driving_log_updated.csv") as f:
    reader = csv.reader(f)
    for line in reader:
        filename = line[0]
        img = cv2.imread(filename)
        img = img[55:-30, :]
        f_split = filename.split("/")
        f_split[-2] = "CROPPED_IMG"
        new_filename = '/'.join(f_split)
        fw.write("{},{},{},{},{}\n".format(new_filename, *line[1:]))
        cv2.imwrite(new_filename, img)
    
fw.close()

### Define Datasets

In [30]:
import random
import csv

TEST_SPLIT_RATIO = 0.2
VALIDATION_SPLIT_RATIO = 0.1

train_data = []
test_data = []
validation_data = []

with open("./data/driving_log_updated_cropped.csv") as f:
    reader = csv.reader(f)
    for line in reader:
        rand = random.random()
        if rand < TEST_SPLIT_RATIO:
            test_data.append(line)
        elif rand < VALIDATION_SPLIT_RATIO+TEST_SPLIT_RATIO:
            validation_data.append(line)
        else:
            train_data.append(line)


In [31]:
total = 1.0* len(train_data)+len(test_data)+len(validation_data)
print(len(train_data), len(train_data)/total )
print(len(test_data), len(test_data)/total)
print(len(validation_data), len(validation_data)/total)


9638 0.7005378688762901
2793 0.2030091583078936
1327 0.09645297281581625


In [35]:
import sklearn

def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        sklearn.utils.shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            vals = np.zeros((batch_size+1, 4))
            imgs = np.zeros((batch_size+1, 75, 320, 3))
            index = 0
            for batch_sample in batch_samples:
                filename = batch_sample[0]
                center_image = cv2.imread(filename)
                imgs[index] = center_image
                index += 1
                try:
                    float(batch_sample[1])
                    float(batch_sample[2])
                    float(batch_sample[3])
                    float(batch_sample[4])
                except:
                    print(batch_sample[1], batch_sample[2],batch_sample[3],batch_sample[4])
                vals[index][0] = batch_sample[1]
                vals[index][1] = batch_sample[2]
                vals[index][2] = batch_sample[3]
                vals[index][3] = batch_sample[4]

            # trim image to only see section with road
            X_train = imgs
            y_train = vals
            yield X_train, y_train


### Define Hyperparameters

In [6]:
epochs = 50
batch_size = 128

train_data_size = len(train_data)
test_data_size = len(test_data)
validation_data_size = len(validation_data)

### Define Model and Train

In [36]:
train_generator = generator(train_data, batch_size=batch_size)
validation_generator = generator(validation_data, batch_size=batch_size)
test_generator = generator(test_data, batch_size=batch_size)

In [8]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Lambda, Flatten
from keras.layers.convolutional import Conv2D

model = Sequential()
model.add(Lambda(lambda x: x/127.5 - 1.0, input_shape=(75, 320, 3)))
model.add(Conv2D(24, (5,5)))
model.add(Conv2D(36, (5,5)))
model.add(Conv2D(48, (5,5)))
model.add(Conv2D(64, (5,5)))
model.add(Conv2D(64, (5,5)))
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(4))


Using TensorFlow backend.


In [9]:
model.compile(optimizer="adam", loss='mean_squared_error', metrics=['accuracy'])

In [ ]:
model.fit_generator(train_generator, samples_per_epoch=int(train_data_size/batch_size), \
                   validation_data=validation_generator, validation_steps=int(validation_data_size/batch_size), \
                   epochs=50)

/Users/wsgreen/miniconda2/envs/dlnd/lib/python3.5/site-packages/ipykernel/__main__.py:1: UserWarning: The semantics of the Keras 2 argument  `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Update your method calls accordingly.
  if __name__ == '__main__':
/Users/wsgreen/miniconda2/envs/dlnd/lib/python3.5/site-packages/ipykernel/__main__.py:1: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., steps_per_epoch=75, validation_steps=10, epochs=50, validation_data=<generator...)`
  if __name__ == '__main__':


Epoch 1/50


### Evaluate

In [ ]:
metrics = evaluate_generator(test_generator, steps=test_data_size/batch_size)

In [ ]:
print(zip(model.metrics_names, metrics))